<a href="https://colab.research.google.com/github/alelorys/chatbot-flask/blob/main/chatBot_flask_testowa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/alelorys/chatbot-flask

Cloning into 'chatbot-flask'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 54 (delta 21), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (54/54), done.


In [2]:
!wget -O - http://download.sgjp.pl/apt/sgjp.gpg.key|sudo apt-key add -
!sudo apt-add-repository http://download.sgjp.pl/apt/ubuntu
!sudo apt update
!sudo apt install morfeusz2 python3-morfeusz2

--2021-11-12 09:02:18--  http://download.sgjp.pl/apt/sgjp.gpg.key
Resolving download.sgjp.pl (download.sgjp.pl)... 193.0.67.154
Connecting to download.sgjp.pl (download.sgjp.pl)|193.0.67.154|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1029 (1.0K) [application/pgp-keys]
Saving to: ‘STDOUT’

-                   100%[===================>]   1.00K  --.-KB/s    in 0s      

2021-11-12 09:02:18 (144 MB/s) - written to stdout [1029/1029]

OK
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [73.0 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://download.sgjp.pl/apt/ubuntu bionic InR

In [3]:
!pip install flask-ngrok
!pip install tflearn

     |████████████████████████████████| 107 kB 28.9 MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=0672787822aed0468741bc39d5af6ca87fe907607060f69f2c63b6045862f0f0
  Stored in directory: /root/.cache/pip/wheels/5f/14/2e/1d8e28cc47a5a931a2fb82438c9e37ef9246cc6a3774520271
Successfully built tflearn


In [50]:
from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok
import json
import pickle
import random
import difflib

import morfeusz2

import nltk
import numpy
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.models import model_from_yaml


#/content/drive/MyDrive/Colabooks/chatBoTest/static
#/content/drive/MyDrive/Colabooks/chatBoTest/templates
TEMPLATE = '/content/chatbot-flask/templates'
STATIC = '/content/chatbot-flask/static'
nltk.download('punkt')

app = Flask(__name__, template_folder=TEMPLATE, static_folder=STATIC)
run_with_ngrok(app)

morf = morfeusz2.Morfeusz()

with open("/content/chatbot-flask/q&a.json") as file:
    data = json.load(file)

try:
    with open("/content/chatbot-flask/chatbot.pickle", "rb") as file:
        words, labels, training, output = pickle.load(file)

except:
    words = []
    labels = []
    docs_x = []
    docs_y = []

    for intent in data["questionAnswer"]:
        for pattern in intent["patterns"]:
            wrds = pattern.split(" ")
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])

        if intent["tag"] not in labels:
            labels.append(intent["tag"])

    Words = []
    for text in words:
      analysis = morf.analyse(text.lower())
      for interpretation in analysis:
        if ":" in interpretation[2][1]:
              s = interpretation[2][1].split(":")
              s.pop()
              s = "".join(s)
              Words.append(s.lower())
        else:
              Words.append(i[2][1].lower())
    words = sorted(list(set(Words)))

    labels = sorted(labels)

    training = []
    output = []

    output_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):
        bag = []

        wrds = []
        for w in doc:
          analysis = morf.analyse(w.lower())
          for interpretation in analysis:
             if ":" in interpretation[2][1]:
              s = interpretation[2][1].split(":")
              s.pop()
              s = "".join(s)
              wrds.append(s.lower())
             else:
              wrds.append(i[2][1].lower())

        wrds = list(set(wrds)) 

        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)

        output_row = output_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)

    training = numpy.array(training)
    output = numpy.array(output)

    with open("/content/chatbot-flask/chatbot.pickle", "wb") as file:
        pickle.dump((words, labels, training, output), file)

try:
    yaml_file = open('/content/chatbot-flask/chatbotmodel.yaml', 'r')
    loaded_model_yaml = yaml_file.read()
    yaml_file.close()
    myChatModel = model_from_yaml(loaded_model_yaml)
    myChatModel.load_weights("/content/chatbot-flask/chatbotmodel.h5")
    

except:
    # Make our neural network
    myChatModel = Sequential()
    myChatModel.add(Dense(16, input_shape=[len(words)], activation='relu'))
    myChatModel.add(Dense(16, activation='relu'))
    myChatModel.add(Dense(len(labels), activation='softmax'))

    # optimize the model
    myChatModel.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # train the model
    myChatModel.fit(training, output, epochs= 60, batch_size = 16, shuffle=True)

    # serialize model to json and save it to disk
    model_json = myChatModel.to_json()
    with open("/content/chatbot-flask/chatbotmodel.json", "w") as j_file:
        j_file.write(model_json)

    # serialize weights to HDF5
    myChatModel.save_weights("/content/chatbot-flask/chatbotmodel.h5")
    print("Saved model from disk")
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = s.split(" ")

    S_words = []

    for word in s_words:
      analysis = morf.analyse(word.lower())
      for interpretation in analysis:
          if ":" in interpretation[2][1]:
              s = interpretation[2][1].split(":")
              s.pop()
              s = "".join(s)
              S_words.append(s.lower())
          else:
              S_words.append(interpretation[2][1].lower())

    S_words = list(set(S_words))
    for se in S_words:
        for i, w in enumerate(words):
          seq = difflib.SequenceMatcher(None,w,se)
          d = seq.ratio()*100
          if w == se:
            bag[i] = 1
          if d >= 75:
            bag[i] = 1
          
        
            
    return numpy.array(bag)

def chatWithBot(userMess):
    currentText = bag_of_words(userMess, words)
    currentTextArray = [currentText]
    numpyCurrentText = numpy.array(currentTextArray)
    if numpy.all((numpyCurrentText == 0)):
      answer = "Nie rozumiem, zadaj pytanie jeszcze raz!"
      return answer

    result = myChatModel.predict(numpyCurrentText[0:1])
    result_index = numpy.argmax(result)
    tag = labels[result_index]
    #if result[0][result_index] > 0.6:
    for tg in data["questionAnswer"]:
      if tg['tag'] == tag:
        responses = tg['responses']

        answer = random.choice(responses)
        return answer

    

def chat(userMess):
    

    while True:
        inp = userMess
        if inp.lower() == "quit":
            break
        if inp.lower() == "" or inp.lower() == " ":
          answer = "Musisz coś napisać!"
          return answer
        
        return chatWithBot(inp)

@app.route("/")
def index():
    return render_template("index.html")

@app.route("/get")
def get_bot_response():
    userMess = request.args.get("msg")
    return chat(userMess)


if __name__ == "__main__":
    app.run()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Epoch 1/60
15/15 [==============================] - 0s 2ms/step - loss: 4.5955 - accuracy: 0.0128
Epoch 2/60
15/15 [==============================] - 0s 2ms/step - loss: 4.5690 - accuracy: 0.0128
Epoch 3/60
15/15 [==============================] - 0s 2ms/step - loss: 4.5469 - accuracy: 0.0214
Epoch 4/60
15/15 [==============================] - 0s 2ms/step - loss: 4.5203 - accuracy: 0.0342
Epoch 5/60
15/15 [==============================] - 0s 2ms/step - loss: 4.4874 - accuracy: 0.0427
Epoch 6/60
15/15 [==============================] - 0s 2ms/step - loss: 4.4447 - accuracy: 0.0556
Epoch 7/60
15/15 [==============================] - 0s 2ms/step - loss: 4.3894 - accuracy: 0.0769
Epoch 8/60
15/15 [==============================] - 0s 2ms/step - loss: 4.3211 - accuracy: 0.0897
Epoch 9/60
15/15 [==============================] - 0s 2ms/step - loss: 4.2377 - accuracy: 0.0983
Epoch 1

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://4a80-35-199-13-206.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [12/Nov/2021 09:46:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Nov/2021 09:46:12] "GET /static/style.css HTTP/1.1" 200 -
127.0.0.1 - - [12/Nov/2021 09:46:12] "GET /static/chat.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Nov/2021 09:46:16] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [12/Nov/2021 09:46:20] "GET /get?msg=Jak%20się%20masz HTTP/1.1" 200 -
127.0.0.1 - - [12/Nov/2021 09:46:54] "GET /get?msg=Jak%20się%20masz%3F HTTP/1.1" 200 -
